In [61]:
import pandas as pd
import csv
import mapreduce as mr
import geopandas as gpd
from shapely.geometry import Point

In [72]:
df = pd.read_csv('vehicle_paths_pnas.csv')

In [73]:
df.head(5)

,Timestamp,Vehicle_ID,Latitude,Longitude,Num_Passengers
0,1380949230,137,40.73467,-73.99459,0
1,1380949230,321,40.78573,-73.97644,0
2,1380949230,428,40.78154,-73.96041,0
3,1380949231,148,40.73478,-73.99871,0
4,1380949231,148,40.73478,-73.99871,0


In [74]:
df_speed = df.loc[df['Num_Passengers'] > 4]

In [75]:
df_speed = df_speed.drop(columns = 'Num_Passengers')

In [76]:
df_speed['Timestamp'] = round(df_speed['Timestamp'] % 86400 / 3600 % 23.5)

In [77]:
def mapper1(_, row):
    #Take out emails send to no enron person
    #Take out emails send to sender his/herself
    #Change the format
    
    yield ([row['Timestamp'], row['Vehicle_ID']], [row['Latitude'], row['Longitude']])

def reducer1(info, location):
    yield(info, location[0])
    
def mapper2(info, location):
    yield (info[0], info[1], location[0], location[1])
            
with open('vehicle_paths_pnas_v2.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output = list(mr.run(mr.run(reader, mapper1, reducer1), mapper2))

In [128]:
df1 = pd.DataFrame(output, columns=['Hour', 'Vehicle_ID', 'Latitude', 'Longitude'])

In [129]:
df1['Latitude'] = df1['Latitude'].astype('float')
df1['Longitude'] = df1['Longitude'].astype('float')

In [130]:
df1 = df1.assign(Location=df1[['Longitude', 'Latitude']].values.tolist())

In [131]:
df1 = df1.drop(['Latitude', 'Longitude'], axis=1)

In [132]:
df1 = df1.drop(['Vehicle_ID'], axis=1)

In [133]:
df1

,Hour,Location
0,0.0,"[-73.99996999999999, 40.73469]"
1,0.0,"[-74.00738, 40.71058]"
2,0.0,"[-73.98695, 40.72099]"
3,0.0,"[-73.99169, 40.7352]"
4,0.0,"[-73.98579000000001, 40.7352]"
5,0.0,"[-74.01151, 40.70297]"
6,0.0,"[-74.01443, 40.71103]"
7,0.0,"[-73.97236, 40.76229]"
8,0.0,"[-74.01324, 40.705870000000004]"
9,0.0,"[-74.00947, 40.710409999999996]"


In [134]:
df1.to_csv('vehicle_paths_pnas_prepared.csv')

In [63]:
df1['Latitude'] = df1['Latitude'].astype('float')
df1['Longitude'] = df1['Longitude'].astype('float')

In [69]:
geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
df = df.drop(['Longitude', 'Latitude'], axis=1)
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

In [71]:
gdf.to_file("e", driver='GeoJSON')